In [41]:
import polars as pl
import sys
import yaml
import os
import pickle
import numpy as np
import pandas as pd

In [42]:
sys.path.append('../src/')
project_root = os.path.dirname(os.getcwd())

In [43]:
# Directorios para los archivos de parámetros y los datos
parameters_directory = os.path.join(project_root, 'src', 'parameters')
data_raw_directory = os.path.join(project_root, 'data', 'raw')
data_processed_directory = os.path.join(project_root, 'data', 'processed')
data_features_directory = os.path.join(project_root, 'data', 'featured')
data_train_directory = os.path.join(project_root, 'data', 'model_input', 'train')
data_val_directory = os.path.join(project_root, 'data', 'model_input', 'validation')
data_test_directory = os.path.join(project_root, 'data', 'model_input', 'test')
data_model_basic_directory = os.path.join(project_root, 'data', 'models', 'basic')
data_model_ensemble_directory = os.path.join(project_root, 'data', 'models', 'ensemble')
data_model_selection_directory = os.path.join(project_root, 'data', 'models', 'model_selection')

In [44]:
# Lista todos los archivos YAML en el directorio especificado
yaml_files = [f for f in os.listdir(parameters_directory) if f.endswith('.yml')]

# Diccionario para guardar los parámetros cargados
parameters = {}

# Carga cada archivo YAML
for yaml_file in yaml_files:
    with open(os.path.join(parameters_directory, yaml_file), 'r') as file:
        data = yaml.safe_load(file)
        key_name = f'parameters_{yaml_file.replace(".yml", "")}'
        parameters[key_name] = data

PROCESSING DATA

In [45]:
# # Nodos
# from functions.processing import (validate_tags_pl, 
#                              validate_dtypes_pl, 
#                              change_names_pl, 
#                              change_dtype_pl,
#                              delete_accents_pl,
#                              standardize_binary_values_pl,
#                              impute_missing_values_pl)

In [46]:
# tag_dict_path = os.path.join(data_raw_directory, parameters['parameters_catalog']['tag_dict_path'])
# raw_data_path = os.path.join(data_raw_directory, parameters['parameters_catalog']['raw_data_path'])
# 
# tag_dict = pl.read_excel(tag_dict_path)
# data_raw = pl.read_csv(raw_data_path)

In [47]:
# prueba1 = validate_tags_pl(data_raw, tag_dict)

In [48]:
# prueba2 = validate_dtypes_pl(prueba1, tag_dict)

In [49]:
# prueba3 = change_names_pl(prueba2, tag_dict)

In [50]:
# prueba4 = change_dtype_pl(prueba3, tag_dict)

In [51]:
# prueba5 = delete_accents_pl(prueba4)

In [52]:
# prueba6 = standardize_binary_values_pl(prueba5, parameters['parameters_processing'])

In [53]:
# prueba7 = impute_missing_values_pl(prueba6)

FEATURING DATA

In [54]:
# # Nodos
# from functions.featuring import (new_features_pl,
#                                  add_target_variable_pl,
#                                  one_hot_encoding_pl,
#                                  random_forest_selection_pl,
#                                  conditional_entropy_selection_pl,
#                                  intersect_top_features_pl)`

In [55]:
# processed_data_path = os.path.join(data_processed_directory, parameters['parameters_catalog']['processed_data_path'])
#
# data_processed = pl.read_csv(processed_data_path)

In [56]:
# prueba8 = new_features_pl(data_processed, parameters['parameters_featuring'])

In [57]:
# target_directory = os.path.join(project_root, 'data', 'processed')
# target_path = os.path.join(target_directory, parameters['parameters_catalog']['target_column_path'])
# target = pl.read_csv(target_path)
# prueba9 = add_target_variable_pl(prueba8, target, parameters['parameters_featuring'])

In [58]:
# prueba10 = one_hot_encoding_pl(prueba9, parameters['parameters_featuring'])

In [59]:
# prueba11 = random_forest_selection_pl(prueba10, parameters['parameters_featuring'])

In [60]:
# prueba12 = conditional_entropy_selection_pl(prueba10, parameters['parameters_featuring'])

In [61]:
# prueba13 = intersect_top_features_pl(prueba11, prueba12, prueba10, parameters['parameters_featuring'])

MODEL INPUT DATA

In [62]:
# # Nodos
# from functions.model_input import (min_max_scaler_pl,
#                                    balance_target_variable_pl,
#                                    train_test_split_pl)

In [63]:
# featured_data_path = os.path.join(data_features_directory, parameters['parameters_catalog']['features_data_path'])
#
# data_featured = pl.read_csv(featured_data_path)

In [64]:
# prueba14 = min_max_scaler_pl(data_featured)

In [65]:
# prueba15 = balance_target_variable_pl(prueba14, parameters['parameters_model_input'])

In [66]:
# prueba16 = train_test_split_pl(prueba15, parameters['parameters_model_input'])

MODELS

In [67]:
# # Nodos
# from functions.models import (train_models_pd)

In [68]:
# train_data_path = os.path.join(data_train_directory, parameters['parameters_catalog']['train_data_path'])
# 
# train_data = pd.read_csv(train_data_path)

In [69]:
# prueba17 = train_models_pd(train_data, parameters['parameters_models'])

In [70]:
# prueba17['basic']['model_name'] 

In [71]:
# prueba17['ensemble']['model_name'] 

MODEL SELECTION

In [72]:
# # Nodos
# from functions.model_selection import (optimize_train_knn, 
#                                        optimize_train_xgboost,
#                                        ab_test_models)

In [73]:
# val_data_path = os.path.join(data_val_directory, parameters['parameters_catalog']['validation_data_path'])
#  
# val_data = pd.read_parquet(val_data_path)

In [74]:
# test_data_path = os.path.join(data_test_directory, parameters['parameters_catalog']['test_data_path'])
#  
# test_data = pd.read_parquet(test_data_path)

In [75]:
# from typing import Dict, Any, Tuple
# def load_model(model_path: str) -> Any:
#     with open(model_path, 'rb') as f:
#         return pickle.loads(f.read())

In [76]:
# basic_model_path = os.path.join(data_model_basic_directory, parameters['parameters_catalog']['basic_model_path'])
# basic_model = load_model(basic_model_path)

In [77]:
# ensemble_model_path = os.path.join(data_model_ensemble_directory, parameters['parameters_catalog']['ensemble_model_path'])
# ensemble_model = load_model(ensemble_model_path)

In [78]:
# prueba18 = optimize_train_knn(basic_model,
#                              val_data,
#                              test_data,
#                              parameters['parameters_model_selection'])

In [79]:
# prueba19 = optimize_train_xgboost(ensemble_model,
#                              val_data,
#                              test_data,
#                              parameters['parameters_model_selection'])

In [80]:
# prueba20 = ab_test_models(prueba18,
#                              prueba19,
#                              test_data,
#                              parameters['parameters_model_selection'])